In [ ]:
#!pip install sentence_transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import stats as st
import seaborn as sns
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

### Чтение 

In [ ]:
pd.set_option('max_colwidth', 500)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
companies = pd.read_excel('https://docs.google.com/spreadsheets/d/1CKM-h3PG2dCspFckUpWbNivuvLh-jiII/export?format=xlsx')

### Исследование

In [ ]:
def general_information(data):
    data.info()
    print()
    display(data.head(20))
    print()
    print('Дубликатов', data.duplicated().sum(), 'или', round(data.duplicated().sum() / len(data) * 100), '%')

In [ ]:
general_information(companies)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3717 entries, 0 to 3716
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   global_id                           3717 non-null   int64 
 1   Наименование поддержанной компании  3717 non-null   object
 2   Отрасль                             3717 non-null   object
 3   Подотрасль                          3717 non-null   object
 4   Технология (1 уровень)              3717 non-null   object
 5   Технология (2 уровень)              3717 non-null   object
 6   Технология (3 уровень)              3717 non-null   object
 7   Сайт                                3717 non-null   object
 8   Описание компании                   3717 non-null   object
dtypes: int64(1), object(8)
memory usage: 261.5+ KB



,global_id,Наименование поддержанной компании,Отрасль,Подотрасль,Технология (1 уровень),Технология (2 уровень),Технология (3 уровень),Сайт,Описание компании
0,784577,"ООО ""АГРОПЛАЗМА""",СЕЛЬСКОЕ ХОЗЯЙСТВО (ВКЛЮЧАЯ РЫБОЛОВСТВО),Растениеводство,Синтез материальных объектов,Синтез живых организмов и систем,Технологии селекции растений,http://www.agroplazma.com,"ООО «Агроплазма» является оригинатором и патентообладателем, занимается производством и продажей семянсобственных сортов сельскохозяйственных культур. Компания «Агроплазма» – первая российская компания, которая на практике осуществляет селекцию сельскохозяйственных растений, связывая полезные признаки (скорость созревания, устойчивость к заболеваниям и др.) с кодирующими генами, комбинируя и отслеживая эти гены в ряду поколений методами молекулярного маркирования (MAS, маркер-опосредованная ..."
1,810784,"ООО ""ЛАБОРАТОРИЯ НАНОСЕМАНТИКА""",ТЕЛЕКОММУНИКАЦИИ И ИНТЕРНЕТ,Деятельность по созданию новых информационных технологий,Синтез информации,Искусственный интеллект,"Обработка естественных языков (перевод, классификация, кластеризация и другое)",https://nanosemantics.ai/,"""Лаборатория Наносемантика"" — ведущий российский разработчик технологий искусственного интеллекта. Компания специализируется на интернет-технологиях и решениях для качественного обслуживания клиентов и разрабатывает виртуальных консультантов, способных имитировать естественную речь. Чат-боты позволяют автоматизировать основную часть взаимодействия с клиентами, привлечь и удержать их внимание, выйти на новую аудиторию и достичь высоких показателей клиентского сервиса. В 2018 году компания зап..."
2,810785,"ООО ""РОМЕТ""",ПРОМЫШЛЕННОСТЬ,"Металлургия, металлопродукция, производство готовых металлических изделий",Синтез материальных объектов,Новые производственные процессы,Технологии сварки и пайки,http://ro-met.com,"Компания ""ROMeT"" специализируется на производстве многослойного металлического материала с протекторной питтинг-защитой. Является инновационным предприятием и в 2011г. поддержана грантом и субсидией Правительства Пензенской области для стимулирования развития инновационной деятельности, коммерциализации научных разработок и успешной реализации инновационных проектов, направленных на разработку многослойных коррозионностойких материалов с протекторной питтинг-защитой.Для реализации проекта ор..."
3,810786,"АО ""СМАРТФИН""",ТЕЛЕКОММУНИКАЦИИ И ИНТЕРНЕТ,Разработка аппаратно-программных комплексов,Обработка и преобразование информации,Обработка и хранение больших данных,Интерфейсы обмена большими данными,https://www.2can.ru,"Компания 2can решает проблему приема денежных средств по банковским картам мобильными продавцами и страховыми агентами с помощью недорогого оборудования (mPOS-ридеров), подключаемых по BlUetooth к смартфону или планшету продавца, а также проблему фискализации в соответствии с 54-ФЗ данных операций на локальных или удаленных кассах продавца. Решение функционирует как сервис мобильного эквайринга, облачная товаро-учетная система и кассовое решение для Android-устройств под брендами 2can и ibox..."
4,810787,"ООО ""УЧИ.РУ""",ТЕЛЕКОММУНИКАЦИИ И ИНТЕРНЕТ,Специализированные информационные системы,Синтез информации,Синтез контента,Синтез образовательного контента,http://www.uchi.ru,"""Учи.ру"" — крупнейшая отечественная образовательная онлайн-платформа для учеников всех возрастов по изучению предметов школьной и внешкольной программы, а также подготовке к ВПР, ОГЭ и ЕГЭ. На цифровой платформе школьники с ранних лет осваивают такие навыки, как: планирование, финансовую грамотность, критическое мышление, знакомятся с программированием и участвуют во всероссийских и международных онлайн-олимпиадах. ""Учи.ру"" — резидент кластера информационных технологий Фонда «Сколково». Сейч..."
5,810788,"ООО ""СДН-ВИДЕО""",ТЕЛЕКОММУНИКАЦИИ И ИНТЕРНЕТ,Деятельность по созданию новых информационных технологий,Обработка и преобразование информации,Обработка и хранение больших данных,Статистическая обработк


Дубликатов 0 или 0 %


### Замена типов

In [ ]:
def type_change(data, column, type): # функция для замены типов данных 
    if type == 'datetime':
        data[column] = pd.to_datetime(data[column])
        print(data[column].dtypes)
    else:
        data[column] = data[column].astype(type, errors='ignore')
        print(data[column].dtypes)

### Преобразование названий столбцов

In [ ]:
companies.columns = companies.columns.str.rstrip()  # убираем пробел в конце строки

In [ ]:
companies.columns = companies.columns.str.replace(' ', '_')

In [ ]:
companies.columns = companies.columns.str.replace('(', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [ ]:
companies.columns = companies.columns.str.replace(')', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [ ]:
general_information(companies)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3717 entries, 0 to 3716
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   global_id                           3717 non-null   int64 
 1   Наименование_поддержанной_компании  3717 non-null   object
 2   Отрасль                             3717 non-null   object
 3   Подотрасль                          3717 non-null   object
 4   Технология_1_уровень                3717 non-null   object
 5   Технология_2_уровень                3717 non-null   object
 6   Технология_3_уровень                3717 non-null   object
 7   Сайт                                3717 non-null   object
 8   Описание_компании                   3717 non-null   object
dtypes: int64(1), object(8)
memory usage: 261.5+ KB



,global_id,Наименование_поддержанной_компании,Отрасль,Подотрасль,Технология_1_уровень,Технология_2_уровень,Технология_3_уровень,Сайт,Описание_компании
0,784577,"ООО ""АГРОПЛАЗМА""",СЕЛЬСКОЕ ХОЗЯЙСТВО (ВКЛЮЧАЯ РЫБОЛОВСТВО),Растениеводство,Синтез материальных объектов,Синтез живых организмов и систем,Технологии селекции растений,http://www.agroplazma.com,"ООО «Агроплазма» является оригинатором и патентообладателем, занимается производством и продажей семянсобственных сортов сельскохозяйственных культур. Компания «Агроплазма» – первая российская компания, которая на практике осуществляет селекцию сельскохозяйственных растений, связывая полезные признаки (скорость созревания, устойчивость к заболеваниям и др.) с кодирующими генами, комбинируя и отслеживая эти гены в ряду поколений методами молекулярного маркирования (MAS, маркер-опосредованная ..."
1,810784,"ООО ""ЛАБОРАТОРИЯ НАНОСЕМАНТИКА""",ТЕЛЕКОММУНИКАЦИИ И ИНТЕРНЕТ,Деятельность по созданию новых информационных технологий,Синтез информации,Искусственный интеллект,"Обработка естественных языков (перевод, классификация, кластеризация и другое)",https://nanosemantics.ai/,"""Лаборатория Наносемантика"" — ведущий российский разработчик технологий искусственного интеллекта. Компания специализируется на интернет-технологиях и решениях для качественного обслуживания клиентов и разрабатывает виртуальных консультантов, способных имитировать естественную речь. Чат-боты позволяют автоматизировать основную часть взаимодействия с клиентами, привлечь и удержать их внимание, выйти на новую аудиторию и достичь высоких показателей клиентского сервиса. В 2018 году компания зап..."
2,810785,"ООО ""РОМЕТ""",ПРОМЫШЛЕННОСТЬ,"Металлургия, металлопродукция, производство готовых металлических изделий",Синтез материальных объектов,Новые производственные процессы,Технологии сварки и пайки,http://ro-met.com,"Компания ""ROMeT"" специализируется на производстве многослойного металлического материала с протекторной питтинг-защитой. Является инновационным предприятием и в 2011г. поддержана грантом и субсидией Правительства Пензенской области для стимулирования развития инновационной деятельности, коммерциализации научных разработок и успешной реализации инновационных проектов, направленных на разработку многослойных коррозионностойких материалов с протекторной питтинг-защитой.Для реализации проекта ор..."
3,810786,"АО ""СМАРТФИН""",ТЕЛЕКОММУНИКАЦИИ И ИНТЕРНЕТ,Разработка аппаратно-программных комплексов,Обработка и преобразование информации,Обработка и хранение больших данных,Интерфейсы обмена большими данными,https://www.2can.ru,"Компания 2can решает проблему приема денежных средств по банковским картам мобильными продавцами и страховыми агентами с помощью недорогого оборудования (mPOS-ридеров), подключаемых по BlUetooth к смартфону или планшету продавца, а также проблему фискализации в соответствии с 54-ФЗ данных операций на локальных или удаленных кассах продавца. Решение функционирует как сервис мобильного эквайринга, облачная товаро-учетная система и кассовое решение для Android-устройств под брендами 2can и ibox..."
4,810787,"ООО ""УЧИ.РУ""",ТЕЛЕКОММУНИКАЦИИ И ИНТЕРНЕТ,Специализированные информационные системы,Синтез информации,Синтез контента,Синтез образовательного контента,http://www.uchi.ru,"""Учи.ру"" — крупнейшая отечественная образовательная онлайн-платформа для учеников всех возрастов по изучению предметов школьной и внешкольной программы, а также подготовке к ВПР, ОГЭ и ЕГЭ. На цифровой платформе школьники с ранних лет осваивают такие навыки, как: планирование, финансовую грамотность, критическое мышление, знакомятся с программированием и участвуют во всероссийских и международных онлайн-олимпиадах. ""Учи.ру"" — резидент кластера информационных технологий Фонда «Сколково». Сейч..."
5,810788,"ООО ""СДН-ВИДЕО""",ТЕЛЕКОММУНИКАЦИИ И ИНТЕРНЕТ,Деятельность по созданию новых информационных технологий,Обработка и преобразование информации,Обработка и хранение больших данных,Статистическая обработка данн


Дубликатов 0 или 0 %


In [ ]:
companies1 = companies.copy()
type_change(companies1, 'Описание_компании', 'string')
type_change(companies1, 'Подотрасль', 'string')
type_change(companies1, 'Технология_3_уровень', 'string')


string
string
string


In [ ]:
companies2 = companies1.drop(['Технология_1_уровень', 'Технология_2_уровень'], axis=1) 

# Поиск несоответсвий

In [ ]:
def compare(desc, cat):
    # Two lists of sentences
    if desc == '' or cat == '':
      return False
    sentences = desc.split('.')

    category = [cat] * len(sentences)

    #Compute embedding for both lists
    embeddings1 = model.encode(sentences, convert_to_tensor=True)
    embeddings2 = model.encode(category, convert_to_tensor=True)

    #Compute cosine-similarities
    cosine_scores = util.cos_sim(embeddings1, embeddings2)

    #Output the pairs with their score
    for i in range(len(sentences)):
        if (cosine_scores[i][i] >= .3):
            return True
    return False


In [ ]:
for i in range(len(companies2)):
  companies2.at[i ,'Сравнение_с_подотрослью'] = compare(companies2['Описание_компании'][i], companies2['Подотрасль'][i])
  companies2.at[i ,'Сравнение_с_технологией_3_уровеня'] = compare(companies2['Описание_компании'][i], companies2['Технология_3_уровень'][i])

In [ ]:
companies2.groupby('Сравнение_с_подотрослью')[['Сравнение_с_подотрослью']].count()

,Сравнение_с_подотрослью
Сравнение_с_подотрослью,
False,280
True,3437


In [ ]:
companies2.groupby('Сравнение_с_технологией_3_уровеня')[['Сравнение_с_технологией_3_уровеня']].count()

,Сравнение_с_технологией_3_уровеня
Сравнение_с_технологией_3_уровеня,
False,609
True,3108


In [ ]:
general_information(companies2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3717 entries, 0 to 3716
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   global_id                           3717 non-null   int64 
 1   Наименование_поддержанной_компании  3717 non-null   object
 2   Отрасль                             3717 non-null   object
 3   Подотрасль                          3717 non-null   string
 4   Технология_3_уровень                3717 non-null   string
 5   Сайт                                3717 non-null   object
 6   Описание_компании                   3717 non-null   string
 7   Сравнение_с_подотрослью             100 non-null    object
 8   Сравнение_с_технологией_3_уровеня   100 non-null    object
dtypes: int64(1), object(5), string(3)
memory usage: 261.5+ KB



,global_id,Наименование_поддержанной_компании,Отрасль,Подотрасль,Технология_3_уровень,Сайт,Описание_компании,Сравнение_с_подотрослью,Сравнение_с_технологией_3_уровеня
0,784577,"ООО ""АГРОПЛАЗМА""",СЕЛЬСКОЕ ХОЗЯЙСТВО (ВКЛЮЧАЯ РЫБОЛОВСТВО),Растениеводство,Технологии селекции растений,http://www.agroplazma.com,"ООО «Агроплазма» является оригинатором и патентообладателем, занимается производством и продажей семянсобственных сортов сельскохозяйственных культур. Компания «Агроплазма» – первая российская компания, которая на практике осуществляет селекцию сельскохозяйственных растений, связывая полезные признаки (скорость созревания, устойчивость к заболеваниям и др.) с кодирующими генами, комбинируя и отслеживая эти гены в ряду поколений методами молекулярного маркирования (MAS, маркер-опосредованная ...",True,True
1,810784,"ООО ""ЛАБОРАТОРИЯ НАНОСЕМАНТИКА""",ТЕЛЕКОММУНИКАЦИИ И ИНТЕРНЕТ,Деятельность по созданию новых информационных технологий,"Обработка естественных языков (перевод, классификация, кластеризация и другое)",https://nanosemantics.ai/,"""Лаборатория Наносемантика"" — ведущий российский разработчик технологий искусственного интеллекта. Компания специализируется на интернет-технологиях и решениях для качественного обслуживания клиентов и разрабатывает виртуальных консультантов, способных имитировать естественную речь. Чат-боты позволяют автоматизировать основную часть взаимодействия с клиентами, привлечь и удержать их внимание, выйти на новую аудиторию и достичь высоких показателей клиентского сервиса. В 2018 году компания зап...",True,False
2,810785,"ООО ""РОМЕТ""",ПРОМЫШЛЕННОСТЬ,"Металлургия, металлопродукция, производство готовых металлических изделий",Технологии сварки и пайки,http://ro-met.com,"Компания ""ROMeT"" специализируется на производстве многослойного металлического материала с протекторной питтинг-защитой. Является инновационным предприятием и в 2011г. поддержана грантом и субсидией Правительства Пензенской области для стимулирования развития инновационной деятельности, коммерциализации научных разработок и успешной реализации инновационных проектов, направленных на разработку многослойных коррозионностойких материалов с протекторной питтинг-защитой.Для реализации проекта ор...",True,True
3,810786,"АО ""СМАРТФИН""",ТЕЛЕКОММУНИКАЦИИ И ИНТЕРНЕТ,Разработка аппаратно-программных комплексов,Интерфейсы обмена большими данными,https://www.2can.ru,"Компания 2can решает проблему приема денежных средств по банковским картам мобильными продавцами и страховыми агентами с помощью недорогого оборудования (mPOS-ридеров), подключаемых по BlUetooth к смартфону или планшету продавца, а также проблему фискализации в соответствии с 54-ФЗ данных операций на локальных или удаленных кассах продавца. Решение функционирует как сервис мобильного эквайринга, облачная товаро-учетная система и кассовое решение для Android-устройств под брендами 2can и ibox...",True,True
4,810787,"ООО ""УЧИ.РУ""",ТЕЛЕКОММУНИКАЦИИ И ИНТЕРНЕТ,Специализированные информационные системы,Синтез образовательного контента,http://www.uchi.ru,"""Учи.ру"" — крупнейшая отечественная образовательная онлайн-платформа для учеников всех возрастов по изучению предметов школьной и внешкольной программы, а также подготовке к ВПР, ОГЭ и ЕГЭ. На цифровой платформе школьники с ранних лет осваивают такие навыки, как: планирование, финансовую грамотность, критическое мышление, знакомятся с программированием и участвуют во всероссийских и международных онлайн-олимпиадах. ""Учи.ру"" — резидент кластера информационных технологий Фонда «Сколково». Сейч...",True,True
5,810788,"ООО ""СДН-ВИДЕО""",ТЕЛЕКОММУНИКАЦИИ И ИНТЕРНЕТ,Деятельность по созданию новых информационных технологий,Статистическая обработка данных,http://cdnvideo.ru,"Компания CDNvideo - оператор сети доставки контента с узлами в 20 городах России, 7 странах СНГ, а также в США, Западной Европе и Юго-Восточной Азии. Услуги CDNvideo служат для ускорения загрузки видео и сайтов. Клиенты - крупнейшие российские телеканалы, О


Дубликатов 0 или 0 %


In [ ]:
companiesFalse = companies2.copy()
companiesFalse = companiesFalse.query('Сравнение_с_подотрослью == False' or 'Сравнение_с_технологией_3_уровеня == False')

In [ ]:
general_information(companiesFalse)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 58 to 95
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   global_id                           4 non-null      int64 
 1   Наименование_поддержанной_компании  4 non-null      object
 2   Отрасль                             4 non-null      object
 3   Подотрасль                          4 non-null      string
 4   Технология_3_уровень                4 non-null      string
 5   Сайт                                4 non-null      object
 6   Описание_компании                   4 non-null      string
 7   Сравнение_с_подотрослью             4 non-null      object
 8   Сравнение_с_технологией_3_уровеня   4 non-null      object
dtypes: int64(1), object(5), string(3)
memory usage: 320.0+ bytes



,global_id,Наименование_поддержанной_компании,Отрасль,Подотрасль,Технология_3_уровень,Сайт,Описание_компании,Сравнение_с_подотрослью,Сравнение_с_технологией_3_уровеня
58,810843,"ООО ""АПСТЕК ЛАБС""",ПРОМЫШЛЕННОСТЬ,Производство электронного и электрического оборудования,Регистрация и преобразование СВЧ и терагерцового излучения,https://navigator.sk.ru/orn/1120774,"После многих лет обширных исследований и разработок ведущими экспертами в области радиофизики, электроники, разработки программного обеспечения, операций по обеспечению безопасности и борьбы с терроризмом компания ООО «АПСТЕК» разработала революционную технологию «HUman SecUrity Radar®» (HSR ™), которая решает одну из самых сложных дилемм безопасности нашего времени: защита уязвимых, многолюдных мест, при этом не задерживая интенсивный поток людей.«HSR™» – первое полностью автоматическое реш...",False,True
90,810876,"ООО ""ГЕОРЕЗОНАНС""",БЕЗОПАСНОСТЬ И КОНТРОЛЬНО-НАДЗОРНАЯ ДЕЯТЕЛЬНОСТЬ,Охрана труда,Генераторы плазмы и системы на их основе,https://www.novas-energy.ru,"Компания ООО «Георезонанс» – резидент инновационного Фонда «Сколково», которая разработала, научно обосновала и защитила российскими и иностранными патентами и применяет на практике технологию Плазменно-импульсного воздействия (ПИВ) для заблаговременной дегазации неразгруженных угольных пластов. Специалисты компании имеют уникальный и многолетний опыт по подбору, монтажу оборудования и эксплуатации вертикальных скважин при заблаговременной дегазации угольных пластов. Только в Кузбассе с 2015...",False,True
92,810878,"ООО ""ВЛАДФОРУМ""",ПРОМЫШЛЕННОСТЬ,Автомобильная промышленность,Хемилюминесцентные методы регистрации,https://forumshop.ru,"Материал «УПТФЭ-ForUm» был изобретен российским ученым, кандидатом химических наук, доцентом – Цветниковым Александром Константиновичем во Владивостоке в конце 80-х и с самого начала задумывался как материал, предназначенный для значительного сокращения затрат на эксплуатацию техники тогда еще советского хозяйства. Ученые Института химии начали самостоятельную работу по разработке линейки продукции на основе нового материала, исходя из потребностей уже современной рыночной системы. В итоге п...",False,True
95,810881,"ООО ""УДАЧНЫЙ ВЫБОР""",ТЕЛЕКОММУНИКАЦИИ И ИНТЕРНЕТ,Деятельность по созданию новых информационных технологий,Средства проектирования и тестирования программного обеспечения,http://www.catapulto.ru,"""CatapUlto – инновационный бесплатный логистический онлайн-сервис, позволяющий в режиме реального времени сравнить динамично меняющиеся цены основных курьерских служб на рынке РФ на конкретные маршруты, необходимые частным и корпоративным клиентам всего за несколько секунд.Компания работает с 2014 года и показывает стабильное географическое и финансовое развитие, ежегодный прирост выручки и постоянное расширение списка партнеров, охватывая 250 стран мира для отправлений своих клиентов. Ежего...",False,True



Дубликатов 0 или 0 %


In [ ]:
companies2.to_excel('Companies.xlsx', index=False)

In [ ]:
companiesFalse.to_excel('Failedcomanies.xlsx', index=False)